In [ ]:
'''
Created on October 282022

@auther: Yan

'''

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor as XGBR
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import log_loss
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import shap
import seaborn as sns
import numpy as np
import pandas as pd
import os



In [ ]:
data = pd.read_excel(r"./ECFP.xlsx")
X = data.iloc[:,3:]
Y = data.iloc[:,2]
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2)
SEED = 1
rfc = RandomForestRegressor(random_state=SEED)
CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
CV_predictions = cross_val_predict(rfc, Xtrain, Ytrain, cv=5)
print(CV_score)
regressor = rfc.fit(Xtrain, Ytrain)
test_predictions = regressor.predict(Xtest)
score_test = regressor.score(Xtest,Ytest)
print(score_test)

In [ ]:
data = pd.read_excel(r"./ECFP.xlsx")
X = data.iloc[:,3:]
Y = data.iloc[:,2]
CV = 0
test =0
for i in range(500):
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2)
    SEED = 1
    rfc = RandomForestRegressor(random_state=SEED)
    CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    CV_predictions = cross_val_predict(rfc, Xtrain, Ytrain, cv=5)
    print("5CV_SCORE:",CV_score)
    regressor = rfc.fit(Xtrain, Ytrain)
    test_predictions = regressor.predict(Xtest)
    score_test = regressor.score(Xtest,Ytest)
    print("TEST_SCORE:",score_test)
#     if CV < CV_score and test < score_test:
#         CV = CV_score
#         test = score_test
    if CV_score>0.70 and score_test>0.70:
        os.mkdir("{}".format(CV_score))
        pd.DataFrame(Xtrain).to_excel('./{}/Xtrain.xlsx'.format(CV_score))
        pd.DataFrame(Ytrain).to_excel('./{}/Ytrain.xlsx'.format(CV_score))
        pd.DataFrame(Xtest).to_excel('./{}/Xtest.xlsx'.format(CV_score))
        pd.DataFrame(Ytest).to_excel('./{}/Ytest.xlsx'.format(CV_score))
        expvspred_5cv = {'Exp': Ytrain, 'Pred':CV_predictions} 
#         pd.DataFrame(expvspred_5cv).to_excel('./Random_forest_5fcv_predictions.xlsx')
        expvspred_test = {'Exp':Ytest,'Pred':test_predictions}
#         pd.DataFrame(expvspred_test).to_excel('./Random_forest_test_predictions.xlsx')
        print("5cv:",CV_score,"TEST:",score_test)
print("5cv:",CV_score,"TEST:",score_test)

In [ ]:
# load Data

data = pd.read_excel(r"./Data_ECFP.xlsx")
Xtrain = np.asarray(data.iloc[:193,2:])
Ytrain = np.asarray(data.iloc[:193,1])
Xtest = np.asarray(data.iloc[193:,2:])
Ytest = np.asarray(data.iloc[193:,1])


In [ ]:
#Model RF

'''

Parameter optimization and five-fold cross validation and parameter optimization

'''

#Random_state

score_5cv_all = []
for i in range(0, 200, 1):
    rfc =RandomForestRegressor(random_state=i)
    score_5cv =cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)

print("Best_5cv score：{}".format(score_max_5cv),
      "random_5cv:{}".format(score_5cv_all.index(score_max_5cv)))

random_state_5cv = range(0, 200)[score_5cv_all.index(max(score_5cv_all))]
print(random_state_5cv)

#N_estimators

score_5cv_all = []
for i in range(1, 400, 1):
    rfc = RandomForestRegressor(n_estimators=i
                                , random_state=random_state_5cv)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
print("Best_5cv score：{}".format(score_max_5cv),
      "n_est_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
n_est_5cv = range(1,400)[score_5cv_all.index(score_max_5cv)]
print(n_est_5cv)

#Max_depth

score_5cv_all = []
for i in range(1, 300, 1):
    rfc = RandomForestRegressor(n_estimators=n_est_5cv
                                ,random_state=random_state_5cv
                                ,max_depth=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
max_depth_5cv = range(1,300)[score_5cv_all.index(score_max_5cv)]
print(max_depth_5cv )

rfc = RandomForestRegressor(n_estimators=n_est_5cv
                            ,random_state=random_state_5cv
                            ,max_depth=max_depth_5cv)
CV_score = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
CV_predictions = cross_val_predict(rfc, Xtrain, Ytrain, cv=5)
rmse = np.sqrt(mean_squared_error(Ytrain,CV_predictions))
print("5cv:",CV_score)
print("rmse_5CV",rmse)
expvspred_5cv = {'Exp': Ytrain, 'Pred':CV_predictions}
pd.DataFrame(expvspred_5cv).to_excel('./Random_forest_5fcv_predictions.xlsx')

'''
Test set validation

'''

rfc = RandomForestRegressor(n_estimators=n_est_5cv
                            ,random_state=random_state_5cv
                            ,max_depth=max_depth_5cv)
regressor = rfc.fit(Xtrain, Ytrain)
test_predictions = regressor.predict(Xtest)
score_test = regressor.score(Xtest,Ytest)
rmse = np.sqrt(mean_squared_error(Ytest,test_predictions))
print("test:",score_test)
print("rmse_test",rmse)
expvspred_test = {'Exp':Ytest,'Pred':test_predictions}
pd.DataFrame(expvspred_test).to_excel('./RF_test_predictions.xlsx')




In [ ]:
def result_figure(y_test_train, pred_train, y_test_test, pred_test, x_label, y_label, data_neme, model_name, path):
    os.makedirs("{}/figure".format(path), exist_ok=True)
    os.makedirs("{}/csv".format(path), exist_ok=True)
    
    r2_train = r2_score(y_test_train, pred_train)
    r2_test = r2_score(y_test_test, pred_test)

    mse_train = MSE(y_test_train, pred_train)
    mse_test = MSE(y_test_test, pred_test)

    mae_train = MAE(y_test_train, pred_train)
    mae_test = MAE(y_test_test, pred_test)
    
    pd.concat([pd.DataFrame(pred_train,columns=["prediction"])
               ,pd.DataFrame(y_test_train,columns=['true'])
               ,pd.DataFrame([r2_train],columns=["r^2"])
               ,pd.DataFrame([np.sqrt(mse_train)],columns=['RMSE'])
               ,pd.DataFrame([mae_train],columns=['MAE'])],axis =1).to_csv("{}/csv/{}_{}_kf.csv".format(path, data_neme, model_name), index=None)
    
    pd.concat([pd.DataFrame(pred_test,columns=["prediction"])
               ,pd.DataFrame(y_test_test,columns=['true'])
               ,pd.DataFrame([r2_test],columns=["r^2"])
               ,pd.DataFrame([np.sqrt(mse_test)],columns=['RMSE'])
               ,pd.DataFrame([mae_test],columns=['MAE'])],axis =1).to_csv("{}/csv/{}_{}_test.csv".format(path, data_neme, model_name), index=None)
    
    plt.figure(figsize=[20,20], dpi=300)
    # plt.grid()
    matplotlib.rcParams['font.family'] = "Arial"
    plt.title('{}_{}'.format(data_neme, model_name),fontsize=70
              ,verticalalignment= 'baseline'
              ,horizontalalignment='center')

    xmax = int(max(list(pred_train)+list(y_test_train)+list(pred_test)+list(y_test_test)))+1
    xmin = int(min(list(pred_train)+list(y_test_train)+list(pred_test)+list(y_test_test)))

    plt.scatter(y_test_train, pred_train,c="royalblue", s=200 ,edgecolor="k",alpha=0.7, label="Train Set")
    plt.scatter(y_test_test, pred_test,c="tomato", s=200 ,edgecolor="k",alpha=0.7, label="Test Set")

    line1 = plt.scatter(-2, -2,c="royalblue", s=1200 ,edgecolor="k",alpha=0.9)
    line2 = plt.scatter(-2, -2,c="tomato", s=1200 ,edgecolor="k",alpha=0.9)

    plt.plot(range(xmin,xmax+1),range(xmin,xmax+1),c='k',linewidth=3, linestyle="--", alpha=0.9)
    plt.xlabel(x_label,fontsize=50)
    plt.ylabel(y_label,fontsize=50, fontdict={"family":"Arial"})
    labels = [""]+[str(i) for i in range(xmin+1,xmax+1)]
    plt.yticks(ticks=range(xmin,xmax+1), labels=labels, fontsize=40)
    plt.xticks(ticks=range(xmin,xmax+1), labels=range(xmin,xmax+1), fontsize=40)
    plt.xlim(xmin,xmax)
    plt.ylim(xmin,xmax)
    plt.legend(fontsize=50, handles=[line1, line2], labels=['Train Set','Test Set'], framealpha=0)
    plt.savefig("{}/figure/{}_{}.png".format(path, data_neme, model_name))
    plt.show()


result_figure(Ytrain, CV_predictions, Ytest, test_predictions, "Actual Values", "Predicted Values", "Presentation_data", "RF", "./results_RF")

In [ ]:
# load Data


data = pd.read_excel(r"./Data_ECFP.xlsx")

Xtrain = np.asarray(data.iloc[:193,2:])
Ytrain = np.asarray(data.iloc[:193,1])
Xtest = np.asarray(data.iloc[193:,2:])
Ytest = np.asarray(data.iloc[193:,1])


#Model XGBoost

'''
Parameter optimization and five-fold cross validation and parameter optimization

'''

#Random_state

score_5cv_all = []
for i in range(0, 200, 1):
    rfc =XGBR(random_state=i)
    score_5cv =cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)

print("Best_5cv score：{}".format(score_max_5cv),
      "random_5cv:{}".format(score_5cv_all.index(score_max_5cv)))

random_state_5cv = range(0, 200)[score_5cv_all.index(max(score_5cv_all))]
print(random_state_5cv)

#Learning_rate

score_5cv_all = []
for i in np.arange(0.01, 0.5, 0.01):
    rfc = XGBR(learning_rate=i
              ,random_state=random_state_5cv)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
print("Best_5cv score：{}".format(score_max_5cv),
      "lr_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
n_lr_5cv = np.arange(0.01,0.5,0.01)[score_5cv_all.index(score_max_5cv)]
print(n_lr_5cv)


#N_estimators

score_5cv_all = []
for i in range(1, 400, 1):
    rfc = XGBR(n_estimators=i
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)

print("Best_5cv score：{}".format(score_max_5cv),
      "n_est_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
n_est_5cv = range(1,400)[score_5cv_all.index(score_max_5cv)]
print(n_est_5cv)

#Max_depth

score_5cv_all = []
for i in range(1, 300, 1):
    rfc = XGBR(n_estimators=n_est_5cv
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    CV_predictions = cross_val_predict(rfc, Xtrain, Ytrain, cv=5)
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)

print("Best_5cv score：{}".format(score_max_5cv),
      "max_depth_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
max_depth_5cv = range(1,300)[score_5cv_all.index(score_max_5cv)]
print(max_depth_5cv )

#Gamma

score_5cv_all = []
for i in np.arange(0,5,0.05):
    rfc = XGBR(n_estimators=n_est_5cv
               ,learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=max_depth_5cv
               ,gamma=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    CV_predictions = cross_val_predict(rfc, Xtrain, Ytrain, cv=5)
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
print("Best_5cv score：{}".format(score_max_5cv),
      "gamma_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
max_gamma_5cv =  np.arange(0,5,0.05)[score_5cv_all.index(score_max_5cv)]
print(max_gamma_5cv)

#Alpha

score_5cv_all = []
for i in np.arange(0,5,0.05):
    rfc = XGBR(n_estimators=n_est_5cv,
               learning_rate=n_lr_5cv
               ,random_state=random_state_5cv
               ,max_depth=max_depth_5cv
               ,gamma=max_gamma_5cv 
               ,alpha=i)
    score_5cv = cross_val_score(rfc, Xtrain, Ytrain, cv=5).mean()
    CV_predictions = cross_val_predict(rfc, Xtrain, Ytrain, cv=5)
    score_5cv_all.append(score_5cv)
    pass
score_max_5cv = max(score_5cv_all)
print("Best_5cv score：{}".format(score_max_5cv),
      "alpha_5cv:{}".format(score_5cv_all.index(score_max_5cv)))
max_alpha_5cv =  np.arange(0,5,0.05)[score_5cv_all.index(score_max_5cv)]
print(max_alpha_5cv)

XGB = XGBR(learning_rate=n_lr_5cv
           ,n_estimators=n_est_5cv
           ,random_state=random_state_5cv
           ,max_depth=max_depth_5cv
           ,gamma =max_gamma_5cv
           ,alpha = max_alpha_5cv)
CV_score = cross_val_score(XGB, Xtrain, Ytrain, cv=5).mean()
CV_predictions = cross_val_predict(XGB, Xtrain, Ytrain, cv=5)
rmse = np.sqrt(mean_squared_error(Ytrain,CV_predictions))
print("5cv:",CV_score)
print("RMSE_5CV",rmse)
expvspred_5cv = {'Exp': Ytrain, 'Pred':CV_predictions}
pd.DataFrame(expvspred_5cv).to_excel('./XGBoost_5fcv_predictions.xlsx')

'''
Test set validation
'''

XGB = XGBR(learning_rate=n_lr_5cv
           ,n_estimators=n_est_5cv
           ,random_state=random_state_5cv
           ,max_depth=max_depth_5cv
           ,gamma =max_gamma_5cv
           ,alpha = max_alpha_5cv)
XGBOOST = XGB.fit(Xtrain, Ytrain)
test_predictions = XGBOOST.predict(Xtest)
score_test = XGBOOST.score(Xtest,Ytest)
rmse = np.sqrt(mean_squared_error(Ytest,test_predictions))
print("test:",score_test)
print("rmse_test",rmse)
expvspred_test = {'Exp':Ytest,'Pred':test_predictions}
pd.DataFrame(expvspred_test).to_excel('./XGB_test_predictions.xlsx')

In [ ]:
result_figure(Ytrain, CV_predictions, Ytest, test_predictions, "Actual Values", "Predicted Values", "Presentation_data", "XGBoost", "./results_xgb")